<a href="https://colab.research.google.com/github/minjundev/Chatting-Generator/blob/main/chatting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

출처 : https://jsideas.net/kakao_rnn/

In [ ]:
!pip install tensorflow==1.14

In [ ]:
import tensorflow as tf
print(tf.__version__)

In [4]:
import glob
import pickle
import pandas as pd

In [5]:
files = glob.glob("./drive/MyDrive/data/*.txt")

In [6]:
def read_das_file(a_file):
    sentence_list = []
    f = open(a_file, 'r', encoding="utf-8")
    for line in f:
        if len(line.split(" ")) == 4:
            if ("https" not in line) & ("사진" not in line) & ("이모티콘" not in line) & ("#" not in line):
                person = line.split(" ")[0][1:-1]

            line = line.split(" ")[3].strip('\n')

            a_dict = {'person': person, 'line': line}
            sentence_list.append(a_dict)
    print("read_das_file 완료")
    return sentence_list

sentence_lists = [read_das_file(file) for file in files]

## 리스트를 평평하게 만든다.
sentence_list = [sentence for sublist in sentence_lists for sentence in sublist]

read_das_file 완료


In [8]:
# 이미 pickle 만들어둠

df = pd.DataFrame(sentence_list)
df['counter'] = 0

counter = 0

## 현재 화자와 다음 화자가 다른 경우, counter를 증가시킨다.
for i in range(1, len(df)):
    current_person = df.loc[i]['person']
    next_person = df.loc[i-1]['person']
    if current_person != next_person:
        counter = counter + 1
    df.at[i, 'counter'] = counter

## 화자와 counter를 더해 message_idx를 만든다.
df['message_idx'] = df.person.astype(str) + "-" + df.counter.astype(str)

## counter와 message_idx로 그룹바이하여 문장을 리스트로 묶는다.
parsed = pd.DataFrame(df.groupby(['counter', 'message_idx'])['line'].apply(list))

## 묶은 라인을 하나의 스트링으로 합치고 줄바꿈을 마지막에 붙인다.
parsed['line2'] = parsed['line'].map(lambda x: " ".join(x).replace("\n", "") + "\n")

## 문장을 pickle로 저장한다.
final_text = parsed['line2'].values
with open("das_data_parsed.pickle", "wb") as f:
    pickle.dump(final_text, f)


In [9]:
# 1. 데이터 준비
import time
from collections import namedtuple
import numpy as np
import tensorflow as tf
import pickle

data = pickle.load(open("./drive/MyDrive/das_data_parsed.pickle", "rb"))
text = "".join(data)

text[:100]

## 사진이 왜케 많냐 저게 다 기본 8년전 7년전 이렇다\n나랑 자주놀앗구먼ㅋㅋ\n싸이월드 사진첩 다운로드 받는중인데 이쁜게 많네\n일시: 5/3(화) 2시 304호  휴. 비도 오


## 모든 문자셋을 만든다.
vocab = set(text)

## 문자셋의 문자에 고유한 숫자를 매긴다. 
vocab_to_int = {c: i for i, c in enumerate(vocab)}

## 그 반대로 숫자에 문자를 대응시킨다. 
## 이렇게 함으로써 신경망을 학습시키고, 
## 신경망에서 생성한 숫자 시퀀스를 다시 문자열로 변환할 수 있다.
int_to_vocab = dict(enumerate(vocab))
chars = np.array([vocab_to_int[c] for c in text], dtype=np.int32)

chars[:100]
## array([841,  509, 1077,  455, ... 1408,  785,  150, 341], dtype=int32)
## 이렇게 문자가 숫자로 변환된다.

num_of_all_chars = np.max(chars) + 1
print("글자 가짓수: %d" %num_of_all_chars)
## 글자 가짓수: 1862

글자 가짓수: 2033


In [10]:
# 2. 학습 & 검증 batch 준비
def split_data(chars, batch_size, num_steps, split_frac=0.9):
    """
    문자 데이터를 학습 & 검증 데이터로 분리
    
    파라미터
    ------
    chars: 문자열 배열
    batch_size: 각 배치 크기
    num_steps: 입력 데이터에 넣을 시퀀스 스텝 수
    split_frac: 트레이닝셋에 넣을 데이터 비율
    
    returns train_x, train_y, val_x, val_y
    """
    
    slice_size = batch_size * num_steps
    n_batches = int(len(chars) / slice_size)
    
    # 배치로 나누고 난 후 남은 잔챙이는 버리자
    # y는 x에 1을 더한다. 왜냐하면 캐릭터 단위로 하나씩 미는 RNN이므로.
    x = chars[: n_batches * slice_size]
    y = chars[1: n_batches * slice_size + 1]
    
    # 배치 크기에 따라 데이터를 자르고, 2차원 매트릭스로 쌓는다.
    x = np.stack(np.split(x, batch_size))
    y = np.stack(np.split(y, batch_size))
    
    # 이제 x와 y는 batch_size x n_batches*num_steps로 된 2차원 배열임
    
    # 이를 학습과 검증 셋으로 나눈다.
    split_idx = int(n_batches * split_frac)
    train_x, train_y = x[:, :split_idx*num_steps], y[:, :split_idx*num_steps]
    val_x, val_y = x[:, split_idx*num_steps:], y[:, split_idx*num_steps:]
    
    return train_x, train_y, val_x, val_y

## 이런식으로 자를 수 있다.
## train_x, train_y, val_x, val_y = split_data(chars, 10, 50)

## 배치를 생성하는 함수를 만든다.
def get_batch(arrs, num_steps):
    """
    함수가 호출될때마다 배치를 리턴한다.
    
    파라미터
    ------
    arrs: 전체 배열
    num_steps: 입력 데이터에 넣을 시퀀스 스텝 수
    """
    batch_size, slice_size = arrs[0].shape
    
    n_batches = int(slice_size / num_steps)
    for b in range(n_batches):
        yield [x[:, b*num_steps: (b+1)*num_steps] for x in arrs]

In [11]:
# 3. RNN 모델 만들기
def build_rnn(num_classes, batch_size=50, num_steps=50, lstm_size=128, num_layers=2,
             learning_rate=0.001, grad_clip=5, sampling=False):
    """
    RNN 모델을 만든다.
    
    파라미터
    ------
    num_classes: 문자 가짓수
    batch_size: 배치 크기
    num_steps: 입력 데이터로 쓸 시퀀스 스텝 수
    lstm_size: LSTM 셀의 유닛 수
    num_layers: LSTM 레이어 갯수
    learning_rate: 학습 속도 알파
    grad_clip: gradient가 폭증하거나 사라지는 것을 막아주는 임계값
    sampling: True인 경우, batch_size와 num_steps를 1로 지정
    """
    
    if sampling == True:
        batch_size, num_steps = 1, 1
        
    # 그래프를 초기화
    tf.reset_default_graph()
    
    # 입력 & 타겟 변수 선언
    inputs = tf.placeholder(tf.int32, [batch_size, num_steps], name='inputs')
    targets = tf.placeholder(tf.int32, [batch_size, num_steps], name='targets')
    
    # dropout 레이어를 위한 값 설정
    keep_prob = tf.placeholder(tf.float32, name='keep_prob')
    
    # 입력 및 타겟 변수를 원핫인코딩
    x_one_hot = tf.one_hot(inputs, num_classes)
    y_one_hot = tf.one_hot(targets, num_classes)
    
    ### RNN 레이어를 만든다
    # 기본 LSTM cell을 사용함
    lstm = tf.contrib.rnn.BasicLSTMCell(lstm_size)
    
    # dropout 레이어 추가
    drop = tf.contrib.rnn.DropoutWrapper(lstm, output_keep_prob=keep_prob)
    
    # LSTM 레이어를 여러개 쌓아서 딥러닝 모델 구축
    cell = tf.contrib.rnn.MultiRNNCell([drop] * num_layers)
    initial_state = cell.zero_state(batch_size, tf.float32)
    
    ### RNN 레이어에 데이터를 넣는다
    rnn_inputs = [tf.squeeze(i, axis=[1]) for i in tf.split(x_one_hot, num_steps, 1)]
    
    # RNN에 각 시퀀스를 넣고 결과를 출력
    outputs, state = tf.contrib.rnn.static_rnn(cell, rnn_inputs, initial_state=initial_state)
    final_state = state
    
    # output의 형태를 바꿈
    seq_output = tf.concat(outputs, axis=1)
    output = tf.reshape(seq_output, [-1, lstm_size])
    
    # RNN 결과를 softmax 레이어에 연결
    with tf.variable_scope('softmax'):
        softmax_w = tf.Variable(tf.truncated_normal((lstm_size, num_classes), stddev=0.1))
        softmax_b = tf.Variable(tf.zeros(num_classes))
    
    # 행렬 계산
    logits = tf.matmul(output, softmax_w) + softmax_b
    
    # 소프트맥스에 넣어 다음 문자의 확률을 계산 (총합은 1)
    preds = tf.nn.softmax(logits, name='prediction')
    
    # target의 형태를 변형해서 logits에 맞춤
    y_reshaped = tf.reshape(y_one_hot, [-1, num_classes])
    loss = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y_reshaped)
    cost = tf.reduce_mean(loss)
    
    # AdamOptimizer와 gradient clipping으로 최적화
    tvars = tf.trainable_variables()
    grads, _ = tf.clip_by_global_norm(tf.gradients(cost, tvars), grad_clip)
    train_op = tf.train.AdamOptimizer(learning_rate)
    optimizer = train_op.apply_gradients(zip(grads, tvars))
    
    # 노드 출력
    export_nodes = ['inputs', 'targets', 'initial_state', 'final_state',
                   'keep_prob', 'cost', 'preds', 'optimizer']
    
    Graph = namedtuple('Graph', export_nodes)
    local_dict = locals()
    graph = Graph(*[local_dict[each] for each in export_nodes])
    
    return graph

In [12]:
# 모델 파라미터 정의
batch_size = 100
num_steps = 100
lstm_size = 512
num_layers = 2
learning_rate = 0.001
keep_prob = 0.5

In [19]:
# 4. 학습
epochs = 50

# N 이터레이션마다 저장
save_every_n = 100

# 데이터 준비
train_x, train_y, val_x, val_y = split_data(chars, batch_size, num_steps)
print(len(vocab))
model = build_rnn(len(vocab),
                 batch_size=batch_size,
                 num_steps=num_steps,
                 learning_rate=learning_rate,
                 lstm_size=lstm_size,
                 num_layers=num_layers)

saver = tf.train.Saver(max_to_keep=100)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    n_batches = int(train_x.shape[1]/num_steps)
    iterations = n_batches * epochs
    
    # 에폭을 돌면서
    for e in range(epochs):
        
        # 네트워크를 학습시킨다
        new_state = sess.run(model.initial_state)
        loss = 0

        # get_batch 함수로 train 데이터셋을 생성하여 모델에 입력한다.
        for b, (x, y) in enumerate(get_batch([train_x, train_y], num_steps), 1):
            iteration = e*n_batches + b
            start = time.time()
            feed = {model.inputs: x,
                    model.targets: y,
                    model.keep_prob: keep_prob,
                    model.initial_state: new_state}
            batch_loss, new_state, _ = sess.run([model.cost, model.final_state, model.optimizer],
                                               feed_dict=feed)
            loss += batch_loss
            end = time.time()
            print('Epoch {}/{}'.format(e+1, epochs),
                 'Iteration {}/{}'.format(iteration, iterations),
                 'Traning loss: {:.4f}'.format(loss/b),
                 '{:.4f} sec/batch'.format((end-start)))
            
            # n 이터레이션 마다, 혹은 마지막 이터레이션에 도달하면
            if (iteration%save_every_n == 0) or (iteration == iterations):
                # 성능을 체크한다.
                val_loss = []
                new_state = sess.run(model.initial_state)
                for x, y in get_batch([val_x, val_y], num_steps):
                    feed = {model.inputs: x,
                            model.targets: y,
                            model.keep_prob: 1.,
                            model.initial_state: new_state}
                    batch_loss, new_state = sess.run([model.cost, model.final_state], feed_dict=feed)
                    val_loss.append(batch_loss)
                    
                print('Validation loss:', np.mean(val_loss), '체크포인트 저장!')
                saver.save(sess, "checkpoints_parsed/i{}_l{}_v{:3f}.ckpt".format(iteration, lstm_size, np.mean(val_loss)))

## Epoch 1/20 Iteration 1/660 Traning loss: 7.5287 8.4289 sec/batch
## Epoch 1/20 Iteration 2/660 Traning loss: 7.4979 7.2635 sec/batch
## Epoch 1/20 Iteration 3/660 Traning loss: 7.3909 7.2701 sec/batch
## ...
## Epoch 3/20 Iteration 99/660 Traning loss: 5.0322 7.1951 sec/batch
## Epoch 4/20 Iteration 100/660 Traning loss: 5.1010 7.1860 sec/batch
## Validation loss: 4.94154 체크포인트 저장!

Epoch 32/50 Iteration 729/1150 Traning loss: 1.7215 10.2100 sec/batch
Epoch 32/50 Iteration 730/1150 Traning loss: 1.7207 10.0790 sec/batch
Epoch 32/50 Iteration 731/1150 Traning loss: 1.7146 9.9890 sec/batch
Epoch 32/50 Iteration 732/1150 Traning loss: 1.7113 10.1040 sec/batch
Epoch 32/50 Iteration 733/1150 Traning loss: 1.7120 10.4899 sec/batch
Epoch 32/50 Iteration 734/1150 Traning loss: 1.7110 9.8694 sec/batch
Epoch 32/50 Iteration 735/1150 Traning loss: 1.7082 10.2435 sec/batch
Epoch 32/50 Iteration 736/1150 Traning loss: 1.7098 9.8747 sec/batch
Epoch 33/50 Iteration 737/1150 Traning loss: 1.7454 10.1248 sec/batch
Epoch 33/50 Iteration 738/1150 Traning loss: 1.7155 10.2601 sec/batch
Epoch 33/50 Iteration 739/1150 Traning loss: 1.7133 10.3562 sec/batch
Epoch 33/50 Iteration 740/1150 Traning loss: 1.7304 10.1535 sec/batch
Epoch 33/50 Iteration 741/1150 Traning loss: 1.7398 10.4078 sec/batch


KeyboardInterrupt: ignored

In [23]:
def pick_top_n(preds, vocab_size, top_n=5):
    p = np.squeeze(preds)
    p[np.argsort(p)[:-top_n]] = 0
    p = p / np.sum(p)
    c = np.random.choice(512, 1, p=p)[0]
    return c

## 체크포인트에 저장된 모델을 불러와서 새로운 텍스트를 생성한다. 
def sample(checkpoint, n_samples, lstm_size, vocab_size, prime="안녕 "):
    samples = [c for c in prime]
    model = build_rnn(len(vocab), lstm_size=lstm_size, sampling=True)
    saver = tf.train.Saver()
    with tf.Session() as sess:
        saver.restore(sess, checkpoint)
        new_state = sess.run(model.initial_state)
        for c in prime:
            x = np.zeros((1, 1))
            x[0, 0] = vocab_to_int[c]
            feed = {model.inputs: x,
                   model.keep_prob: 1.,
                   model.initial_state: new_state}
            preds, new_state = sess.run([model.preds, model.final_state],
                                       feed_dict=feed)
            
        c = pick_top_n(preds, len(vocab))
        samples.append(int_to_vocab[c])
        
        for i in range(n_samples):
            x[0, 0] = c
            feed = {model.inputs:x,
                   model.keep_prob: 1.,
                   model.initial_state: new_state}
            
            preds, new_state = sess.run([model.preds, model.final_state],
                                       feed_dict=feed)
            
            c = pick_top_n(preds, len(vocab))
            samples.append(int_to_vocab[c])
            
        return ''.join(samples)

## 이런식으로 모델을 불러와서 실행한다.
checkpoint = "./checkpoints_parsed/i700_l512_v1.311343.ckpt"
samp = sample(checkpoint, 100, lstm_size, len(vocab), prime="경")
print(samp)

INFO:tensorflow:Restoring parameters from ./checkpoints_parsed/i700_l512_v1.311343.ckpt
경이이이이이?이가진?이이진이이?진이이이진이진진이나이이?이이가?가이이진이이이나이?나진나진??이이진나이가이진나나진이진이이??진이진가진가진이가?이?이이이이이진이??이진이가이이?진?이이이이나
